In [5]:
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

In [17]:
def read_file_numbers(filename):
    filepath = os.path.join('Data', f'{filename}.txt')

    try:
        numbers = []

        with open(filepath, 'r') as file:
            for line in file:
                line = line.strip()

                if not line:
                    continue

                try:
                    number = float(line)
                    numbers.append(number)
                except ValueError:
                    continue

        return np.array(numbers)

    except FileNotFoundError:
        return np.array([])
    except Exception:
        return np.array([])


def variance_threshold_array(data, threshold, kernel_size=5):
    result = np.zeros(len(data))
    half_kernel = kernel_size // 2
    in_below_threshold_region = False
    current_value = 0

    for i in range(len(data)):
        start = max(0, i - half_kernel)
        end = min(len(data), i + half_kernel + 1)

        window = data[start:end]
        variance = np.var(window)

        if variance <= threshold:
            if not in_below_threshold_region:
                current_value = float(input("Enter value for new region below threshold: "))
                in_below_threshold_region = True
            result[i] = current_value
        else:
            result[i] = 0
            in_below_threshold_region = False

    return result


def write_array_to_file(array, filename):
    filepath = os.path.join('Data', filename)

    try:
        with open(filepath, 'w') as file:
            for value in array:
                file.write(f"{value}\n")
        return True
    except Exception:
        return False


import matplotlib.pyplot as plt
import numpy as np


def plot_array(array, title="Gráfico do Array", xlabel="Índice", ylabel="Valores",
               color='blue', linewidth=1.5, figsize=(12, 6), margin=0.1):
    if not isinstance(array, np.ndarray):
        array = np.array(array)

    plt.figure(figsize=figsize)
    plt.plot(range(len(array)), array, color=color, linewidth=linewidth)

    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.grid(True, alpha=0.3)

    y_min, y_max = np.min(array), np.max(array)
    y_range = y_max - y_min

    if y_range < 1e-10:
        y_margin = 0.01 * abs(y_min) if y_min != 0 else 0.01
    else:
        y_margin = margin * y_range

    plt.ylim(y_min - y_margin, y_max + y_margin)

    x_margin = margin * len(array)
    plt.xlim(-x_margin, len(array) - 1 + x_margin)

    plt.tight_layout()
    plt.show()


def add_y_noise(array, range, same_noise=False):
    if not isinstance(array, np.ndarray):
        array = np.array(array)

    result = array.copy()

    if same_noise:
        noise_value = range
        result += noise_value
    else:
        noise = np.random.uniform(0 - (range / 2.0), 0 + (range / 2.0), len(array))
        result += noise

    return result


def create_range_skip_zero(start, end, steps):
    all_values = np.linspace(start, end, steps + 1)
    zero_idx = np.argmin(np.abs(all_values))

    values = np.delete(all_values, zero_idx)
    return values[:steps]


def shift_array(array, shift_amount, noise_range=0.01):
    if not isinstance(array, np.ndarray):
        array = np.array(array)

    result = np.zeros_like(array)

    if shift_amount == 0:
        return array.copy()

    elif shift_amount > 0:
        result[shift_amount:] = array[:-shift_amount]
        first_value = array[0]
        noise = np.random.uniform(0 - (noise_range / 2.0), 0 + (noise_range / 2.0), shift_amount)
        result[:shift_amount] = first_value + noise

    else:
        shift_amount = abs(shift_amount)
        result[:-shift_amount] = array[shift_amount:]
        last_value = array[-1]
        noise = np.random.uniform(0 - (noise_range / 2.0), 0 + (noise_range / 2.0), shift_amount)
        result[-shift_amount:] = last_value + noise

    return result

def fault_array(array, fault_size):
    max_start_index = len(array) - fault_size
    fault_start = random.randint(0, max_start_index)
    fault_end = fault_start + fault_size

    faulty_array = array.copy()
    faulty_array[fault_start:fault_end] = 0

    return faulty_array

In [ ]:
# array = read_file_numbers("BK7-Cu")
# array_label = variance_threshold_array(array, 0.00005, 25)
#
# write_array_to_file(array_label, "BK7-Cu-Label.txt")

In [16]:
array = read_file_numbers("BK7-Au")

y_indexes = create_range_skip_zero(-0.1, 0.1, 20)
np.random.shuffle(y_indexes)

noise_indexes = create_range_skip_zero(0.0, 0.025, 10)
np.random.shuffle(noise_indexes)

shift_indexes = np.linspace(1, 25, 25)
shift_indexes = shift_indexes.astype(int)
np.random.shuffle(shift_indexes)

final = []

for i in y_indexes:
    for j in noise_indexes:
        for w in shift_indexes:
            modified = add_y_noise(array, i, same_noise=True)
            modified = add_y_noise(modified, j, same_noise=False)
            modified = shift_array(modified, w)

            final.append(modified)

array = np.save('data_BK7_Au_5000.npy', final)

NameError: name 'read_file_numbers' is not defined

In [18]:
array = read_file_numbers("BK7-Au")

y_indexes = create_range_skip_zero(-0.1, 0.1, 20)
np.random.shuffle(y_indexes)

noise_indexes = create_range_skip_zero(0.0, 0.025, 10)
np.random.shuffle(noise_indexes)

shift_indexes = np.linspace(1, 25, 25)
shift_indexes = shift_indexes.astype(int)
np.random.shuffle(shift_indexes)

final = []
output = []

for i in y_indexes:
    for j in noise_indexes:
        for w in shift_indexes:
            modified = add_y_noise(array, i, same_noise=True)
            modified = add_y_noise(modified, j, same_noise=False)
            modified = shift_array(modified, w)

            output.append(modified)

min = np.min(output)
max = np.max(output)

print(np.min(output), np.max(output))
output = (output - np.min(output)) / (np.max(output) - np.min(output))
print(np.min(output), np.max(output))

for i in output:
    modified = fault_array(i, 120)
    final.append(modified)

X = np.array(final)
Y = np.array(output)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

np.save("CNN/x_train.npy", x_train)
np.save("CNN/x_test.npy", x_test)
np.save("CNN/y_train.npy", y_train)
np.save("CNN/y_test.npy", y_test)

67.94850653842494 69.3144910646539
0.0 1.0


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, Dense, Flatten, Reshape, MaxPooling1D, UpSampling1D, BatchNormalization, Activation, Dropout, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

model = tf.keras.models.load_model('best_model_val_mae.h5')

x_test = np.load('CNN/x_test.npy')
y_test = np.load('CNN/y_test.npy')

# Avaliar o modelo com o conjunto de teste
test_loss, test_mae = model.evaluate(x_test, y_test, verbose=1)
print(f"Perda no teste: {test_loss}")
print(f"MAE no teste: {test_mae}")

# Fazer previsões em alguns exemplos do conjunto de teste
n_samples = 3
test_array = x_test[:n_samples]
predictions = model.predict(test_array)

# Visualizar as previsões comparadas com os valores reais
plt.figure(figsize=(15, 10))
for i in range(n_samples):
    plt.subplot(n_samples, 1, i + 1)

    plt.plot((y_test[i] * (max - min)) + min, label='Real', alpha=1.0)
    plt.plot((predictions[i] * (max - min)) + min, label='Previsão', alpha=1.0)
    plot_test_array = np.copy(test_array[i])
    plot_test_array[plot_test_array == 0] = np.nan
    plt.plot((plot_test_array * (max - min)) + min, label='Input', alpha=1.0)
    plt.title(f'Amostra {i + 1}')
    plt.legend()

plt.tight_layout()
plt.show()

16/16 [==============================] - 7s 36ms/step - loss: 0.0148 - mae: 0.0449
Perda no teste: 0.014831281267106533
MAE no teste: 0.04493958130478859
1/1 [==============================] - 0s 374ms/step


FileNotFoundError: [Errno 2] No such file or directory: 'cnn_scaler.gz'

In [ ]:
array = read_file_numbers("BK7-Au")

shift_y = add_y_noise(array, 0.1, same_noise=True)
noise = add_y_noise(array, 0.025, same_noise=False)
shift_x = shift_array(array, 25)

fig, axs = plt.subplots(2, 1, figsize=(16, 9), dpi=200)

axs[0].plot(array)
axs[0].set_title('Array Original')
axs[0].grid(True, linestyle='--', alpha=0.6)
#
# axs[1].plot(shift_y, 'r')
# axs[1].set_title('Deslocamento Vertical (shift_y)')
# axs[1].grid(True, linestyle='--', alpha=0.6)

axs[1].plot(noise, 'g')
axs[1].set_title('Ruído Aleatório Adicionado (noise)')
axs[1].grid(True, linestyle='--', alpha=0.6)
#
# axs[3].plot(shift_x, 'm')
# axs[3].set_title('Elementos Deslocados/Rolados (shift_x)')
# axs[3].grid(True, linestyle='--', alpha=0.6)

fig.text(0.5, 0.04, 'Índice do Array', ha='center', va='center', fontsize=12)

plt.tight_layout(rect=[0, 0.05, 1, 0.96])

plt.savefig('colagem_coluna_alta_resolucao.png', dpi=300)

plt.show()

In [ ]:
# --- Geração dos Arrays de Dados ---
# 1. Array original
array = read_file_numbers("BK7-Au")

# 2. Array com deslocamento vertical (ruído constante)
shift_y = add_y_noise(array, 0.1, same_noise=True)

# 3. Array com ruído aleatório
noise = add_y_noise(array, 0.025, same_noise=False)

# 4. Array com elementos deslocados (rolados)
shift_x = shift_array(array, 25)


# --- Criação do Gráfico Único ---
# Define o tamanho da figura
plt.figure(figsize=(12, 7), dpi=200)

# Plota cada array no mesmo gráfico, especificando cor e rótulo
plt.plot(array,   color='blue',   label='Array Original')
plt.plot(shift_y, color='red',    label='Deslocamento Vertical', linestyle='--')
plt.plot(shift_x, color='green',    label='Deslocamento Horizontal', linestyle='--')

# Adiciona títulos e rótulos aos eixos
plt.xlabel('Índice do Array', fontsize=12)
plt.ylabel('Valor', fontsize=12)

# Adiciona uma grade para melhor visualização
plt.grid(True, linestyle='--', alpha=0.6)

# Adiciona a legenda para identificar cada linha
plt.legend(fontsize=10)

# Ajusta o layout para garantir que tudo fique visível
plt.tight_layout()


# --- Salvando a Imagem em Alta Resolução ---
# Salva a figura final com 300 PPI (pontos por polegada)
plt.savefig('grafico_unico_alta_resolucao.png', dpi=300)

# Exibe o gráfico (opcional)
plt.show()